In [51]:
import requests
import pandas as pd
import json
from dateutil import parser

In [3]:
API_KEY = "667e94661abb533b7c59e065eed32b7a-e8470dd11ead1617b105cb8843647d6c"
ACCOUNT_ID = "101-001-25318071-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [4]:
session = requests.Session()

In [6]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [12]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [17]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [18]:
response = session.get(url, params=None, data=None, headers=None)

In [19]:
response.status_code

200

In [20]:
data = response.json()

In [24]:
instruments_list = data['instruments']

In [25]:
len(instruments_list)

68

In [27]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [28]:
key_inst = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [30]:
instruments_dict = {}

for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_inst}

In [32]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [33]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [71]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
        )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()

    if response.status_code == 200:
        if "candles" not in data:
            data = []
        else: 
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ["mid", "bid", "ask"]
    ohlc = ["o", "h", "l", "c"]

    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']

        for p in prices:
            for o in ohlc:
                new_dict[f'{p}_{o}'] = float(candle[p][o])
        
        final_data.append(new_dict)

    df = pd.DataFrame.from_dict(final_data)
    return df

In [72]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [74]:
candles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    9 non-null      datetime64[ns, tzutc()]
 1   volume  9 non-null      int64                  
 2   mid_o   9 non-null      float64                
 3   mid_h   9 non-null      float64                
 4   mid_l   9 non-null      float64                
 5   mid_c   9 non-null      float64                
 6   bid_o   9 non-null      float64                
 7   bid_h   9 non-null      float64                
 8   bid_l   9 non-null      float64                
 9   bid_c   9 non-null      float64                
 10  ask_o   9 non-null      float64                
 11  ask_h   9 non-null      float64                
 12  ask_l   9 non-null      float64                
 13  ask_c   9 non-null      float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), in